1. Based on the dataframe created in feature engineering 2, left join with the orders_new table (you just need column user_id and eval_set) 
where the orders are not from the prior eval_set. 

2. Split the dataset into two, one for train and one for test based on column eval_set

3. Left join with order_products_new table where eval_set is train, the reordered column will be the target varaible, 
set the value for reordered column to be 0 for products that are not ordered in the most recent order.

4. Apply Xgboost model on the dataset - please use cv for parameter tuning, and apply the final model to test dataset)

#### Based on the dataframe created in feature engineering 2, left join with the orders_new table (you just need column user_id and eval_set)
#### where the orders are not from the prior eval_set.

In [1]:
import pandas as pd
import numpy as np

In [2]:
orders_new = pd.read_csv('orders_new.csv')

In [3]:
data = pd.read_csv('data.csv')

In [4]:
orders_new_part = orders_new.filter(['user_id','order_id','eval_set'],axis=1)

In [5]:
orders_new_part = orders_new_part.loc[orders_new_part['eval_set'] !='prior']

In [6]:
a = orders_new_part[orders_new_part['eval_set'] == 'train']
a.head()

,user_id,order_id,eval_set
3214874,1,1187899,train
3214875,2,1492625,train
3214878,5,2196797,train
3214880,7,525192,train
3214881,8,880375,train


In [7]:
b = orders_new_part[orders_new_part['eval_set'] == 'test']
b.head()

,user_id,order_id,eval_set
3214876,3,2774568,test
3214877,4,329954,test
3214879,6,1528013,test
3214884,11,1376945,test
3214885,12,1356845,test


In [8]:
ab = a.append(b, ignore_index=True)

In [9]:
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   206209 non-null  int64 
 1   order_id  206209 non-null  int64 
 2   eval_set  206209 non-null  object
dtypes: int64(2), object(1)
memory usage: 4.7+ MB


In [10]:
combined = pd.merge(data,ab, how="left", on="user_id")


In [11]:
combined.head()

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,...,prod_second_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,order_id,eval_set
0,1,196,10,1,10,1.400000,10,176.0,17.6,59,...,4660.0,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000,1187899,train
1,1,10258,9,2,10,3.333333,10,176.0,17.6,59,...,308.0,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000,1187899,train
2,1,10326,1,5,5,5.000000,10,176.0,17.6,59,...,1003.0,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667,1187899,train
3,1,12427,10,1,10,3.300000,10,176.0,17.6,59,...,889.0,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000,1187899,train
4,1,13032,3,2,10,6.333333,10,176.0,17.6,59,...,617.0,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333,1187899,train


In [13]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 25 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   user_id                          int64  
 1   product_id                       int64  
 2   up_orders                        int64  
 3   up_first_order                   int64  
 4   up_last_order                    int64  
 5   up_average_cart_position         float64
 6   user_orders                      int64  
 7   sum_days_since_prior_order       float64
 8   avg_days_since_prior_order       float64
 9   total_products                   int64  
 10  distinct_products                int64  
 11  reordered_ratio                  float64
 12  prod_orders                      int64  
 13  prod_reorders                    int64  
 14  prod_first_orders                float64
 15  prod_second_orders               float64
 16  prod_reorder_probability         float64
 17  prod_r

In [14]:
combined['eval_set'].unique()

array(['train', 'test'], dtype=object)

#### 2 Split the dataset into two, one for train and one for test based on column eval_set

In [12]:
train = combined.loc[combined['eval_set'] == 'train']
train.head()

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,...,prod_second_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,order_id,eval_set
0,1,196,10,1,10,1.400000,10,176.0,17.6,59,...,4660.0,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000,1187899,train
1,1,10258,9,2,10,3.333333,10,176.0,17.6,59,...,308.0,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000,1187899,train
2,1,10326,1,5,5,5.000000,10,176.0,17.6,59,...,1003.0,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667,1187899,train
3,1,12427,10,1,10,3.300000,10,176.0,17.6,59,...,889.0,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000,1187899,train
4,1,13032,3,2,10,6.333333,10,176.0,17.6,59,...,617.0,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333,1187899,train


In [13]:
test = combined.loc[combined['eval_set'] =='test']
test.head()

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,...,prod_second_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,order_id,eval_set
120,3,248,1,2,2,3.000000,12,133.0,11.083333,88,...,1068.0,0.279508,1.667365,0.400251,7.333333,0.083333,10,0.090909,2774568,test
121,3,1005,1,10,10,5.000000,12,133.0,11.083333,88,...,81.0,0.312741,1.787645,0.440605,7.333333,0.083333,2,0.333333,2774568,test
122,3,1819,3,4,7,2.666667,12,133.0,11.083333,88,...,427.0,0.346872,1.969131,0.492162,7.333333,0.250000,5,0.333333,2774568,test
123,3,7503,1,3,3,6.000000,12,133.0,11.083333,88,...,2137.0,0.383731,2.239899,0.553551,7.333333,0.083333,9,0.100000,2774568,test
124,3,8021,1,2,2,5.000000,12,133.0,11.083333,88,...,5022.0,0.440836,2.445927,0.591157,7.333333,0.083333,10,0.090909,2774568,test


#### 3 Left join with order_products_new table where eval_set is train, the reordered column will be the target varaible, 
####          set the value for reordered column to be 0 for products that are not ordered in the most recent order.

In [17]:
order_products_new = pd.read_csv('order_products_new.csv')

order_products_new.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [ ]:
train_new = pd.merge(order_products_new, train, how='right', on=['order_id','product_id'])

In [16]:
train_new.fillna(0)

,order_id,product_id,add_to_cart_order,reordered,user_id,up_orders,up_first_order,up_last_order,up_average_cart_position,user_orders,...,prod_first_orders,prod_second_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,eval_set
0,1187899,196,1.0,1.0,1,10,1,10,1.400000,10,...,8000.0,4660.0,0.582500,4.473875,0.776480,5.900000,1.000000,0,1.000000,train
1,1187899,10258,6.0,1.0,1,9,2,10,3.333333,10,...,557.0,308.0,0.552962,3.493716,0.713772,5.900000,0.900000,0,1.000000,train
2,1187899,10326,0.0,0.0,1,1,5,5,5.000000,10,...,1923.0,1003.0,0.521581,2.873635,0.652009,5.900000,0.100000,5,0.166667,train
3,1187899,12427,0.0,0.0,1,10,1,10,3.300000,10,...,1679.0,889.0,0.529482,3.857058,0.740735,5.900000,1.000000,0,1.000000,train
4,1187899,13032,7.0,1.0,1,3,2,10,6.333333,10,...,1286.0,617.0,0.479782,2.916796,0.657158,5.900000,0.300000,0,0.333333,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8474656,272231,43961,0.0,0.0,206209,3,4,12,8.000000,13,...,20455.0,10527.0,0.514642,2.706967,0.630583,9.923077,0.230769,1,0.300000,train
8474657,272231,44325,0.0,0.0,206209,1,7,7,8.000000,13,...,2087.0,605.0,0.289890,1.669861,0.401148,9.923077,0.076923,6,0.142857,train
8474658,272231,48370,0.0,0.0,206209,1,11,11,8.000000,13,...,1183.0,701.0,0.592561,3.325444,0.699288,9.923077,0.076923,2,0.333333,train
8474659,272231,48697,0.0,0.0,206209,1,7,7,6.000000,13,...,6284.0,1638.0,0.260662,1.556811,0.357661,9.923077,0.076923,6,0.142857,train


In [ ]:
joint = pd.merge(train_new, order_products_new, on = 'order_id', how = 'left')
joint = joint.loc[:,['user_id', 'product_id', 'reordered']]

In [ ]:
train_set = pd.merge(train, joint, how = 'left', on = ['user_id','product_id'])

In [ ]:
train_set = train_set.drop(columns=['eval_set'])
test_set = test.drop(columns=['eval_set'])

In [ ]:
train_set = train_set.fillna(0)
test_set = test_set.fillna(0)

train_features = train_set.iloc[:,:-1]
train_target = train_set.iloc[:,-1]

### Establish train and test dataset

In [59]:
from sklearn.model_selection import train_test_split


In [64]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2)

### kfold

In [ ]:
def train_validation_split(data, k):
    number_of_rows = data.shape[0]
    number_of_test = int(np.floor(number_of_rows / k))
    list_of_data_index = [i for i in range(number_of_rows)]
    
    result = pd.DataFrame(columns=['train_index','validation_index'], index=[i for i in range(k)])
    
    for i in range(k):
        total_index = np.array([i for i in range(number_of_rows)])
        test_index = np.random.choice(list_of_data_index, size=number_of_test, replace=False)
        train_index = np.delete(total_index, test_index)
        
        for index in test_index:
            list_of_data_index.remove(index)
        
        # print("{0}th iteration:\n".format({i}), "test index\n", test_index, "\ntrain index\n", train_index)
        
        result.iloc[i,0] = np.sort(train_index)
        result.iloc[i,1] = np.sort(test_index)

    return result

In [ ]:
def cv(model, x_train, y_train, k):
    # get kfold index
    index = kfold_index(x_train, k)
    #train each sub-combination
    auc = []
    for i in range(k):
        cv_xtest = x_train.iloc[index[i][0]:index[i][1],:]
        cv_ytest = y_train.iloc[index[i][0]:index[i][1],]
        # drop the test sub-set
        cv_xtrain = x_train.drop(x_train.index[index[i][0]:index[i][1]])
        cv_ytrain = y_train.drop(y_train.index[index[i][0]:index[i][1]])
        model.fit(cv_xtrain,cv_ytrain.values.reshape(-1))
        y_pred = model.predict(cv_xtest)
        # calculate the auc as the evaluation metric
        fpr, tpr, threshold = roc_curve(cv_ytest, rtree.predict_proba(cv_xtest)[:,1])
        aa = metrics.auc(fpr, tpr)
        auc.append(aa)
    return np.mean(auc)

#### 4 Apply Xgboost model on the dataset - please use cv for parameter tuning, and apply the final model to test dataset

In [65]:
import xgboost as xgb
from xgboost import XGBRegressor

In [70]:
xgb = XGBRegressor(n_estimators=500, learning_rate=0.05, min_child_weight=5, max_depth=4)

In [ ]:
xgb.fit(X_train,Y_train)

In [ ]:
print("Validation:", xgb.score(x_valid,y_valid))

predict = xgb.predict(x_valid)

In [ ]:
from sklearn import metrics
preds_train = bst.predict(x_train,x)
metrics.r2_score(y_train.values, pred_train)

In [86]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn import model_selection, metrics
from sklearn.model_selection import GridSearchCV

In [87]:
def modelfit(alg, x_train, y_train, useTrainCV=True, cv_fold=5,early_stopping_rounds =50):
    alg.fit(x_train,y_train,eval_metric='auc')  #model
    train_predictions = alg.predict(x_train)   #predict the train dataset
    print("\n about this model")
    print("r2_score : %.4g" % metrics.r2_score(y_train.values, train_predictions))

In [ ]:
xgb1 = XGBRegressor(base_score=0.5,
                   colsample_bylevel=1,
                   colsample_bytree=1,
                   gamma=0,
                   learning_rate=0.1,
                   max_delta_step=0,
                   max_depth=3,
                   min_child_weight=1,
                   missing=None,
                   n_estimators=100,
                   nthread=-1,
                   objective='reg:linear',
                   reg_alpha=0,
                   reg_lambda=1,
                   scale_pos_weight=1,
                   seed=1850,
                   silent=True,
                   subsample=1)
modelfit(xgb, x_train,y_train)

In [89]:
#定义评价函数，可以传入后面模型中替代模型本身的损失函数
def rmspe(y,yhat):
    return np.sqrt(np.mean((yhat/y-1)**2))

def rmspe_xg(yhat,y):
    y=np.expm1(y.get_label())
    yhat=np.expm1(yhat)
    return 'rmspe',rmspe(y,yhat)

In [ ]:
params={
       'objective':'reg:linear',
       'booster':'gbtree',
       'eta':0.03,
       'max_depth':10,
       'subsample':0.9,
       'colsample_bytree':0.7,
       'silent':1,
       'seed':10}
num_boost_round=6000

dtrain=xgb.DMatrix(x_train,y_train)
dvalid=xgb.DMatrix(x_test,y_test)
watchlist=[(dtrain,'train'),(dvalid,'eval')]


print('Train a XGBoost model')
start=time()
gbm=xgb.train(params,dtrain,num_boost_round,evals=watchlist,
             early_stopping_rounds=100,feval=rmspe_xg,verbose_eval=True)
end=time()

print('Train time is {:.2f} s.'.format(end-start))


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from searchgrid import set_grid, make_grid_search
from sklearn.model_selection import GridSearchCV

In [ ]:
train_new['reordered'] = train_new['add_to_cart_order'].apply(lambda x: 1 if x == train_new['add_to_cart_order'] == idxmax() else 0)

a['eval_set'] = a['product_id'].apply(lambda x: 'test' if x == 0 else 'train' )

In [ ]:
data = combined.drop(combined.loc[combined.groupby('user_id')['order_number'].idxmax()].index)